# 130: ML Observability & Debugging

## 🎯 Learning Objectives

By the end of this notebook, you will:
- **Understand** distributed tracing for ML pipelines (track data → feature → model → prediction flow)
- **Implement** model explainability with SHAP and LIME (debug individual predictions)
- **Build** performance profiling systems (identify latency bottlenecks in inference pipelines)
- **Apply** error analysis techniques to post-silicon validation (root cause detection for test failures)
- **Master** debugging strategies for production ML systems (systematic troubleshooting)
- **Deploy** comprehensive observability dashboards (unified view of ML system health)

## 📚 What is ML Observability?

**ML Observability** is the practice of monitoring, understanding, and debugging machine learning systems in production through comprehensive instrumentation and analysis.

Unlike traditional software observability (logs, metrics, traces), ML observability adds:
- **Model performance tracking:** Accuracy, latency, drift over time
- **Prediction explainability:** Why did the model make this specific prediction?
- **Feature attribution:** Which features contributed most to the prediction?
- **Error analysis:** What patterns exist in model failures?
- **Data quality monitoring:** Is input data within expected distributions?

**Traditional Observability:**
```
Request → Server → Database → Response
   ↓         ↓         ↓          ↓
 Trace    Logs    Metrics    Status
```

**ML Observability:**
```
Request → Feature Engineering → Model Inference → Prediction → Response
   ↓              ↓                    ↓              ↓           ↓
 Trace      Feature Values      SHAP Values    Confidence    Status
   ↓              ↓                    ↓              ↓           ↓
Input Data   Missing Features   Latency Breakdown  Accuracy   Errors
```

**Why ML Observability?**
- ✅ **Debug faster:** Identify root cause of prediction errors in minutes (not days)
- ✅ **Prevent incidents:** Detect anomalies before they impact business (early warning)
- ✅ **Explain decisions:** Provide transparency for stakeholders (regulatory compliance)
- ✅ **Optimize performance:** Identify bottlenecks, reduce latency by 50-80%
- ✅ **Improve models:** Learn from failures, prioritize retraining efforts

## 🏭 Post-Silicon Validation Use Cases

**Use Case 1: Wafer Test Failure Root Cause Analysis**
- **Input:** 1000 failed devices (binning model predicted fail, actual fail)
- **Output:** Top 3 root causes (Vdd out of range → 45%, spatial correlation → 30%, temperature → 25%)
- **Value:** Debug time reduced from 8 hours (manual inspection) → 15 minutes (automated analysis)

**Use Case 2: Test Time Optimization Debugging**
- **Input:** Test time prediction model (predicting 45ms, actual 120ms for specific devices)
- **Output:** Bottleneck identified (feature: device_complexity underestimated by 60%)
- **Value:** Fix feature engineering bug → improve prediction RMSE from 30ms → 8ms

**Use Case 3: Binning Model Explainability**
- **Input:** Device binned as "Fail" (customer disputes, demands explanation)
- **Output:** SHAP waterfall plot (Vdd contribution: -0.3, Idd: -0.2, frequency: -0.1 → total: -0.6 fail score)
- **Value:** Regulatory compliance (IEEE 1505 audit trail), customer transparency

**Use Case 4: Spatial Correlation Model Performance Profiling**
- **Input:** Wafer map inference taking 500ms (SLA: <100ms)
- **Output:** Latency breakdown (neighbor search: 400ms, feature compute: 80ms, model: 20ms)
- **Value:** Optimize neighbor search (spatial index) → reduce latency 500ms → 60ms

## 🔄 ML Observability Workflow

```mermaid
graph TB
    A[Production ML System] --> B[Distributed Tracing]
    A --> C[Model Explainability]
    A --> D[Performance Profiling]
    A --> E[Error Analysis]
    
    B --> F[Feature → Model → Prediction Flow]
    C --> G[SHAP/LIME Analysis]
    D --> H[Latency Breakdown]
    E --> I[Root Cause Detection]
    
    F --> J[Observability Dashboard]
    G --> J
    H --> J
    I --> J
    
    J --> K[Alerts & Insights]
    K --> L[Debug & Fix]
    L --> A
    
    style A fill:#e1f5ff
    style J fill:#ffe1e1
    style L fill:#e1ffe1
```

## 📊 Learning Path Context

**Prerequisites:**
- **Notebook 129:** Advanced MLOps - Feature Stores & Real-Time Monitoring (drift detection, data quality)
- **Notebook 128:** Shadow Mode Deployment (A/B testing, canary deployment)
- **Notebook 127:** ML Governance & Compliance (audit trails, lineage tracking)

**Next Steps:**
- **Notebook 131:** Containerization for ML (Docker, Kubernetes, model serving)
- **Notebook 132:** Service Mesh for ML (Istio, traffic management, observability)
- **Notebook 133:** CI/CD for ML (automated testing, deployment pipelines)

---

Let's build production-grade ML observability and debugging systems! 🚀

## 2. 🔍 Distributed Tracing for ML Pipelines

### 📝 What's Happening in This Section?

**Purpose:** Implement distributed tracing to track the complete flow of ML predictions from request → feature engineering → model inference → response, capturing timing, metadata, and errors at each stage.

**Key Points:**
- **Trace ID propagation**: Single ID follows request through entire pipeline (correlate all operations)
- **Span hierarchy**: Parent-child relationships (request → feature_fetch → model_predict → response)
- **Timing instrumentation**: Capture start/end timestamps for each operation (identify bottlenecks)
- **Context enrichment**: Attach metadata (feature values, model version, input size, cache hits)
- **Error tracking**: Capture exceptions with full context (which stage failed, why)

**Why This Matters:**
- **Debug production issues:** "Why did this specific prediction take 2 seconds?" (answer in trace)
- **Optimize latency:** Identify slowest operation (feature fetch: 1.5s → optimize caching)
- **Root cause analysis:** Trace errors back to source (missing feature → upstream pipeline failure)

**Post-Silicon Application:** Trace wafer binning prediction: STDF ingestion → feature engineering (spatial correlation) → model inference → binning decision (track end-to-end flow, identify delays)

In [ ]:
@dataclass
class Span:
    """Individual operation in distributed trace"""
    span_id: str
    parent_span_id: Optional[str]
    operation_name: str
    start_time: float
    end_time: Optional[float] = None
    metadata: Dict[str, Any] = field(default_factory=dict)
    error: Optional[str] = None
    
    @property
    def duration_ms(self) -> float:
        """Calculate span duration in milliseconds"""
        if self.end_time:
            return (self.end_time - self.start_time) * 1000
        return 0.0
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert span to dictionary for logging"""
        return {
            'span_id': self.span_id,
            'parent_span_id': self.parent_span_id,
            'operation': self.operation_name,
            'duration_ms': round(self.duration_ms, 2),
            'metadata': self.metadata,
            'error': self.error
        }


class MLTracer:
    """Distributed tracing system for ML pipelines"""
    
    def __init__(self):
        self.traces = {}  # trace_id → list of spans
        self.active_spans = {}  # span_id → Span
    
    def start_trace(self, trace_id: str, operation_name: str) -> str:
        """
        Start new trace (root span)
        
        Args:
            trace_id: Unique identifier for this request
            operation_name: Name of root operation (e.g., "predict_request")
        
        Returns:
            span_id: ID of root span
        """
        span_id = f"{trace_id}_span_0"
        span = Span(
            span_id=span_id,
            parent_span_id=None,
            operation_name=operation_name,
            start_time=time.time()
        )
        
        self.traces[trace_id] = [span]
        self.active_spans[span_id] = span
        
        return span_id
    
    def start_span(self, trace_id: str, parent_span_id: str, operation_name: str, 
                   metadata: Optional[Dict] = None) -> str:
        """
        Start child span within trace
        
        Args:
            trace_id: ID of parent trace
            parent_span_id: ID of parent span
            operation_name: Name of this operation
            metadata: Optional metadata to attach
        
        Returns:
            span_id: ID of new span
        """
        span_count = len(self.traces.get(trace_id, []))
        span_id = f"{trace_id}_span_{span_count}"
        
        span = Span(
            span_id=span_id,
            parent_span_id=parent_span_id,
            operation_name=operation_name,
            start_time=time.time(),
            metadata=metadata or {}
        )
        
        if trace_id not in self.traces:
            self.traces[trace_id] = []
        
        self.traces[trace_id].append(span)
        self.active_spans[span_id] = span
        
        return span_id
    
    def end_span(self, span_id: str, metadata: Optional[Dict] = None, error: Optional[str] = None):
        """
        End span, record duration and optional metadata/error
        
        Args:
            span_id: ID of span to end
            metadata: Additional metadata to attach
            error: Error message if operation failed
        """
        if span_id in self.active_spans:
            span = self.active_spans[span_id]
            span.end_time = time.time()
            
            if metadata:
                span.metadata.update(metadata)
            
            if error:
                span.error = error
            
            del self.active_spans[span_id]
    
    def get_trace(self, trace_id: str) -> List[Dict[str, Any]]:
        """Get all spans for a trace"""
        if trace_id in self.traces:
            return [span.to_dict() for span in self.traces[trace_id]]
        return []
    
    def get_trace_summary(self, trace_id: str) -> Dict[str, Any]:
        """Get summary statistics for trace"""
        spans = self.traces.get(trace_id, [])
        
        if not spans:
            return {}
        
        total_duration = max(span.duration_ms for span in spans)
        operation_durations = {}
        
        for span in spans:
            op = span.operation_name
            if op not in operation_durations:
                operation_durations[op] = []
            operation_durations[op].append(span.duration_ms)
        
        # Calculate percentage breakdown
        operation_breakdown = {
            op: {
                'total_ms': sum(durations),
                'percentage': (sum(durations) / total_duration * 100) if total_duration > 0 else 0,
                'count': len(durations)
            }
            for op, durations in operation_durations.items()
        }
        
        # Check for errors
        errors = [span.error for span in spans if span.error]
        
        return {
            'trace_id': trace_id,
            'total_duration_ms': round(total_duration, 2),
            'span_count': len(spans),
            'operation_breakdown': operation_breakdown,
            'errors': errors
        }


# Example: Trace wafer binning prediction pipeline
print("=" * 60)
print("Distributed Tracing for Wafer Binning Pipeline")
print("=" * 60)

tracer = MLTracer()

# Simulate 5 predictions with tracing
for i in range(5):
    trace_id = f"wafer_predict_{i:03d}"
    
    # Root span: prediction request
    root_span = tracer.start_trace(trace_id, "predict_wafer_binning")
    
    # Span 1: Fetch features from feature store
    fetch_span = tracer.start_span(
        trace_id, root_span, "fetch_features",
        metadata={'wafer_id': f'W{i:04d}', 'feature_count': 15}
    )
    time.sleep(0.01 + np.random.uniform(0, 0.02))  # Simulate variable latency
    tracer.end_span(fetch_span, metadata={'cache_hit': i % 2 == 0})
    
    # Span 2: Compute spatial correlation features
    spatial_span = tracer.start_span(
        trace_id, root_span, "compute_spatial_features",
        metadata={'neighbor_count': 24, 'radius_mm': 3}
    )
    time.sleep(0.005 + np.random.uniform(0, 0.015))
    tracer.end_span(spatial_span, metadata={'neighbors_found': 24})
    
    # Span 3: Model inference
    model_span = tracer.start_span(
        trace_id, root_span, "model_inference",
        metadata={'model_version': 'v2.3', 'model_type': 'RandomForest'}
    )
    time.sleep(0.003 + np.random.uniform(0, 0.007))
    tracer.end_span(model_span, metadata={'prediction': 'Pass' if i % 3 != 0 else 'Fail'})
    
    # End root span
    tracer.end_span(root_span, metadata={'response_status': 200})

# Analyze traces
print("\n📊 Trace Analysis:")
print("-" * 60)

for i in range(5):
    trace_id = f"wafer_predict_{i:03d}"
    summary = tracer.get_trace_summary(trace_id)
    
    print(f"\nTrace ID: {summary['trace_id']}")
    print(f"  Total Duration: {summary['total_duration_ms']:.2f} ms")
    print(f"  Operations:")
    
    for op, stats in sorted(summary['operation_breakdown'].items(), 
                           key=lambda x: x[1]['percentage'], reverse=True):
        print(f"    - {op}: {stats['total_ms']:.2f} ms ({stats['percentage']:.1f}%)")

# Identify bottleneck
print("\n🎯 Performance Bottleneck Analysis:")
print("-" * 60)

all_summaries = [tracer.get_trace_summary(f"wafer_predict_{i:03d}") for i in range(5)]
avg_breakdown = {}

for summary in all_summaries:
    for op, stats in summary['operation_breakdown'].items():
        if op not in avg_breakdown:
            avg_breakdown[op] = []
        avg_breakdown[op].append(stats['percentage'])

print("\nAverage Time Breakdown:")
for op, percentages in sorted(avg_breakdown.items(), 
                             key=lambda x: np.mean(x[1]), reverse=True):
    avg_pct = np.mean(percentages)
    print(f"  {op}: {avg_pct:.1f}% of total time")

print("\n✅ Bottleneck identified: Optimize 'fetch_features' operation (caching, batching)")
print("✅ Trace IDs enable correlating errors across distributed services")

## 3. 🧠 Model Explainability and Debugging with SHAP

### 📝 What's Happening in This Section?

**Purpose:** Implement SHAP (SHapley Additive exPlanations) values to explain individual predictions, debug model behavior, and identify which features contributed most to specific decisions.

**Key Points:**
- **SHAP values**: Unified measure of feature importance based on game theory (Shapley values)
- **Additive feature attribution**: Prediction = base_value + Σ(SHAP_values) (exact decomposition)
- **Local explanations**: Why this specific prediction? (waterfall plot shows feature contributions)
- **Global explanations**: Which features matter most overall? (summary plot aggregates across dataset)
- **Model-agnostic**: Works for any model (tree-based, neural networks, linear models)

**Why This Matters:**
- **Debug predictions:** "Why did model predict Fail for device X?" (Vdd=-0.3, Idd=-0.2 → -0.5 total)
- **Build trust:** Stakeholders understand model reasoning (not black box)
- **Feature engineering:** Identify uninformative features (remove noise, improve performance)
- **Regulatory compliance:** Provide audit trail for decisions (IEEE 1505, FDA requirements)

**Post-Silicon Application:** Explain wafer binning decisions: Device binned as "Fail" - SHAP shows Vdd contributed -0.3 (primary driver), spatial correlation -0.15 (neighboring devices also failed), temperature -0.05 (minor factor)

In [ ]:
class SimpleSHAPExplainer:
    """
    Simplified SHAP-like explainer for tree-based models
    
    Note: This is an educational implementation. For production, use the `shap` library:
    import shap
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    """
    
    def __init__(self, model, X_background: np.ndarray, feature_names: List[str]):
        """
        Initialize explainer
        
        Args:
            model: Trained model with predict_proba method
            X_background: Background dataset for computing baseline
            feature_names: Names of features
        """
        self.model = model
        self.X_background = X_background
        self.feature_names = feature_names
        self.base_value = model.predict_proba(X_background)[:, 1].mean()
    
    def explain_instance(self, X_instance: np.ndarray, num_samples: int = 100) -> Dict[str, float]:
        """
        Compute approximate SHAP values for single instance using permutation
        
        Args:
            X_instance: Single data point to explain (1D array)
            num_samples: Number of permutations for approximation
        
        Returns:
            Dictionary mapping feature names to SHAP values
        """
        n_features = len(X_instance)
        shap_values = np.zeros(n_features)
        
        # Get prediction for instance
        pred = self.model.predict_proba(X_instance.reshape(1, -1))[0, 1]
        
        # Approximate SHAP values using permutation importance
        for i in range(n_features):
            # Create modified instances with feature i from background
            marginal_contrib = []
            
            for _ in range(num_samples):
                # Random background sample
                bg_idx = np.random.randint(0, len(self.X_background))
                X_modified = X_instance.copy()
                X_modified[i] = self.X_background[bg_idx, i]
                
                # Prediction difference
                pred_modified = self.model.predict_proba(X_modified.reshape(1, -1))[0, 1]
                marginal_contrib.append(pred - pred_modified)
            
            shap_values[i] = np.mean(marginal_contrib)
        
        # Normalize to sum to (prediction - base_value)
        total_shap = shap_values.sum()
        target_sum = pred - self.base_value
        
        if abs(total_shap) > 1e-6:
            shap_values = shap_values * (target_sum / total_shap)
        
        return dict(zip(self.feature_names, shap_values))
    
    def plot_waterfall(self, shap_values: Dict[str, float], instance_prediction: float, 
                      instance_data: Dict[str, Any], title: str = "SHAP Waterfall Plot"):
        """
        Create waterfall plot showing feature contributions
        
        Args:
            shap_values: Dictionary of feature SHAP values
            instance_prediction: Model prediction for this instance
            instance_data: Feature values for this instance
            title: Plot title
        """
        # Sort features by absolute SHAP value
        sorted_features = sorted(shap_values.items(), key=lambda x: abs(x[1]), reverse=True)
        
        # Take top 10 features
        top_features = sorted_features[:10]
        
        # Create waterfall data
        features = ['Base Value'] + [f[0] for f in top_features] + ['Prediction']
        values = [self.base_value] + [f[1] for f in top_features] + [instance_prediction]
        
        # Calculate cumulative sum for plotting
        cumulative = [self.base_value]
        for _, shap_val in top_features:
            cumulative.append(cumulative[-1] + shap_val)
        cumulative.append(instance_prediction)
        
        # Plot
        fig, ax = plt.subplots(figsize=(12, 6))
        
        colors = ['blue' if v >= 0 else 'red' for v in [0] + [f[1] for f in top_features] + [0]]
        
        for i in range(len(features) - 1):
            if i == 0:
                ax.barh(i, cumulative[i], color='gray', alpha=0.3)
            else:
                start = cumulative[i-1]
                width = values[i]
                ax.barh(i, width, left=start, color=colors[i], alpha=0.7)
                
                # Add connecting line
                if i < len(features) - 1:
                    ax.plot([cumulative[i], cumulative[i]], [i-0.4, i+0.4], 
                           'k--', linewidth=0.5, alpha=0.3)
        
        # Final prediction bar
        ax.barh(len(features)-1, cumulative[-1], color='green', alpha=0.3)
        
        # Labels
        ax.set_yticks(range(len(features)))
        feature_labels = ['Base Value']
        for fname, shap_val in top_features:
            fval = instance_data.get(fname, 'N/A')
            feature_labels.append(f"{fname}={fval:.3f}\n({shap_val:+.3f})")
        feature_labels.append(f"Prediction\n{instance_prediction:.3f}")
        ax.set_yticklabels(feature_labels, fontsize=9)
        
        ax.set_xlabel('Model Output (Probability)', fontsize=11)
        ax.set_title(title, fontsize=13, fontweight='bold')
        ax.grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plt.show()


# Example: Explain wafer binning predictions
print("=" * 60)
print("Model Explainability: Wafer Device Binning")
print("=" * 60)

# Generate synthetic STDF-like data
np.random.seed(42)
n_samples = 1000

# Features: Vdd, Idd, frequency, temperature, test_time, spatial_correlation
data = {
    'vdd': np.random.normal(1.2, 0.02, n_samples),
    'idd': np.random.normal(100, 10, n_samples),
    'frequency': np.random.normal(2000, 100, n_samples),
    'temperature': np.random.normal(25, 5, n_samples),
    'test_time_ms': np.random.normal(50, 10, n_samples),
    'neighbor_yield_avg': np.random.uniform(0.7, 1.0, n_samples)
}

df = pd.DataFrame(data)

# Create target: Fail if Vdd too low OR neighbor yield low
df['binning'] = (
    ((df['vdd'] < 1.18) | (df['neighbor_yield_avg'] < 0.75))
).astype(int)

# Train model
X = df[['vdd', 'idd', 'frequency', 'temperature', 'test_time_ms', 'neighbor_yield_avg']].values
y = df['binning'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)

accuracy = accuracy_score(y_test, model.predict(X_test))
print(f"\n✅ Model trained: {accuracy:.2%} accuracy on test set")

# Create explainer
feature_names = ['vdd', 'idd', 'frequency', 'temperature', 'test_time_ms', 'neighbor_yield_avg']
explainer = SimpleSHAPExplainer(model, X_train, feature_names)

print(f"📊 Base prediction (average): {explainer.base_value:.3f}")

# Explain specific failed device
failed_indices = np.where(y_test == 1)[0]
if len(failed_indices) > 0:
    fail_idx = failed_indices[0]
    X_fail = X_test[fail_idx]
    
    print("\n" + "=" * 60)
    print("Explaining Failed Device Prediction")
    print("=" * 60)
    
    # Get prediction
    pred_prob = model.predict_proba(X_fail.reshape(1, -1))[0, 1]
    pred_class = model.predict(X_fail.reshape(1, -1))[0]
    
    print(f"\nDevice Features:")
    instance_data = {}
    for i, fname in enumerate(feature_names):
        print(f"  {fname}: {X_fail[i]:.3f}")
        instance_data[fname] = X_fail[i]
    
    print(f"\nPrediction: {'Fail' if pred_class == 1 else 'Pass'} (probability: {pred_prob:.3f})")
    
    # Compute SHAP values
    print("\n🔍 Computing SHAP values (feature attributions)...")
    shap_vals = explainer.explain_instance(X_fail, num_samples=50)
    
    print("\nFeature Contributions (SHAP values):")
    for fname, shap_val in sorted(shap_vals.items(), key=lambda x: abs(x[1]), reverse=True):
        direction = "→ FAIL" if shap_val > 0 else "→ PASS"
        print(f"  {fname}: {shap_val:+.4f} {direction}")
    
    # Verify: base_value + sum(SHAP) ≈ prediction
    total_shap = sum(shap_vals.values())
    reconstructed = explainer.base_value + total_shap
    print(f"\n✅ Verification:")
    print(f"  Base value: {explainer.base_value:.4f}")
    print(f"  Sum of SHAP values: {total_shap:+.4f}")
    print(f"  Predicted probability: {pred_prob:.4f}")
    print(f"  Reconstructed: {reconstructed:.4f} (difference: {abs(pred_prob - reconstructed):.6f})")
    
    # Visualize
    explainer.plot_waterfall(shap_vals, pred_prob, instance_data, 
                            title="SHAP Waterfall: Why Device Failed Binning")

print("\n" + "=" * 60)
print("Key Insights:")
print("-" * 60)
print("• SHAP values show exact contribution of each feature to prediction")
print("• Negative SHAP values push toward Pass (class 0)")
print("• Positive SHAP values push toward Fail (class 1)")
print("• Use for debugging: Identify which features drove wrong predictions")
print("• Use for transparency: Explain decisions to stakeholders")

## 4. ⚡ Performance Profiling and Latency Optimization

### 📝 What's Happening in This Section?

**Purpose:** Build performance profiling system to measure latency breakdown of ML inference pipelines, identify bottlenecks, and optimize for production SLAs (<100ms p99 latency).

**Key Points:**
- **Latency breakdown**: Measure time spent in each stage (data preprocessing: 40ms, model inference: 30ms, post-processing: 20ms)
- **Percentile analysis**: Track p50, p95, p99 latency (catch tail latency issues that impact UX)
- **Batch vs single inference**: Compare throughput (single: 50 QPS, batch=32: 800 QPS)
- **Memory profiling**: Track peak memory usage (detect memory leaks, optimize batch size)
- **CPU/GPU utilization**: Identify underutilized resources (GPU at 30% → increase batch size)

**Why This Matters:**
- **Meet SLAs:** Production requires <100ms p99 latency (profiling reveals 150ms → optimize to 80ms)
- **Cost optimization:** Increase throughput 16x (50 QPS → 800 QPS) by batching (fewer servers needed)
- **User experience:** Reduce tail latency (p99: 500ms → 120ms) improves customer satisfaction
- **Capacity planning:** Understand resource limits (max throughput before latency degrades)

**Post-Silicon Application:** Profile wafer map inference: Spatial neighbor search takes 400ms (80% of latency) → optimize with KD-tree → reduce to 50ms → total latency 500ms → 100ms (5x speedup)

In [ ]:
class PerformanceProfiler:
    """Performance profiling system for ML inference pipelines"""
    
    def __init__(self):
        self.stage_timings = {}  # stage_name → list of durations
        self.memory_usage = []
    
    def profile_stage(self, stage_name: str):
        """Context manager for profiling a stage"""
        return StageProfiler(self, stage_name)
    
    def record_timing(self, stage_name: str, duration_ms: float):
        """Record timing for a stage"""
        if stage_name not in self.stage_timings:
            self.stage_timings[stage_name] = []
        self.stage_timings[stage_name].append(duration_ms)
    
    def get_statistics(self) -> Dict[str, Dict[str, float]]:
        """Compute summary statistics for all stages"""
        stats = {}
        
        for stage, timings in self.stage_timings.items():
            if timings:
                stats[stage] = {
                    'count': len(timings),
                    'mean_ms': np.mean(timings),
                    'std_ms': np.std(timings),
                    'p50_ms': np.percentile(timings, 50),
                    'p95_ms': np.percentile(timings, 95),
                    'p99_ms': np.percentile(timings, 99),
                    'min_ms': np.min(timings),
                    'max_ms': np.max(timings)
                }
        
        return stats
    
    def print_summary(self, sla_ms: Optional[float] = None):
        """Print performance summary"""
        stats = self.get_statistics()
        
        print("\n" + "=" * 80)
        print("Performance Profile Summary")
        print("=" * 80)
        
        print(f"\n{'Stage':<30} {'Count':>8} {'Mean':>10} {'P50':>10} {'P95':>10} {'P99':>10}")
        print("-" * 80)
        
        total_mean = 0
        for stage, s in sorted(stats.items(), key=lambda x: x[1]['mean_ms'], reverse=True):
            print(f"{stage:<30} {s['count']:>8} {s['mean_ms']:>9.2f}ms {s['p50_ms']:>9.2f}ms "
                  f"{s['p95_ms']:>9.2f}ms {s['p99_ms']:>9.2f}ms")
            total_mean += s['mean_ms']
        
        print("-" * 80)
        print(f"{'TOTAL':<30} {'':<8} {total_mean:>9.2f}ms")
        
        if sla_ms:
            # Check p99 against SLA
            max_p99 = max(s['p99_ms'] for s in stats.values())
            sla_status = "✅ PASS" if max_p99 < sla_ms else "❌ FAIL"
            print(f"\nSLA Check (p99 < {sla_ms}ms): {sla_status} (actual p99: {max_p99:.2f}ms)")
    
    def plot_latency_distribution(self):
        """Plot latency distribution for each stage"""
        stats = self.get_statistics()
        
        if not stats:
            print("No timing data to plot")
            return
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Plot 1: Box plot of latencies
        ax1 = axes[0]
        stage_names = list(self.stage_timings.keys())
        data = [self.stage_timings[stage] for stage in stage_names]
        
        bp = ax1.boxplot(data, labels=stage_names, patch_artist=True)
        for patch in bp['boxes']:
            patch.set_facecolor('skyblue')
        
        ax1.set_ylabel('Latency (ms)', fontsize=11)
        ax1.set_title('Latency Distribution by Stage', fontsize=12, fontweight='bold')
        ax1.grid(axis='y', alpha=0.3)
        plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        # Plot 2: Cumulative percentage breakdown
        ax2 = axes[1]
        total_times = {stage: np.sum(timings) for stage, timings in self.stage_timings.items()}
        total = sum(total_times.values())
        
        sorted_stages = sorted(total_times.items(), key=lambda x: x[1], reverse=True)
        stages = [s[0] for s in sorted_stages]
        percentages = [(s[1] / total * 100) for s in sorted_stages]
        
        colors = plt.cm.Set3(np.linspace(0, 1, len(stages)))
        ax2.bar(stages, percentages, color=colors, alpha=0.8)
        
        ax2.set_ylabel('% of Total Time', fontsize=11)
        ax2.set_title('Time Breakdown by Stage', fontsize=12, fontweight='bold')
        ax2.grid(axis='y', alpha=0.3)
        plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        plt.tight_layout()
        plt.show()


class StageProfiler:
    """Context manager for profiling individual stages"""
    
    def __init__(self, profiler: PerformanceProfiler, stage_name: str):
        self.profiler = profiler
        self.stage_name = stage_name
        self.start_time = None
    
    def __enter__(self):
        self.start_time = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        duration_ms = (time.time() - self.start_time) * 1000
        self.profiler.record_timing(self.stage_name, duration_ms)


# Example: Profile wafer map inference pipeline
print("=" * 60)
print("Performance Profiling: Wafer Map Inference")
print("=" * 60)

profiler = PerformanceProfiler()

# Train model on synthetic wafer data
np.random.seed(42)
n_devices = 5000

X_wafer = np.random.randn(n_devices, 10)
y_wafer = (X_wafer[:, 0] + X_wafer[:, 1] > 0).astype(int)

X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(
    X_wafer, y_wafer, test_size=0.2, random_state=42
)

wafer_model = RandomForestClassifier(n_estimators=50, max_depth=8, random_state=42)
wafer_model.fit(X_train_w, y_train_w)

print("✅ Model trained for wafer map inference")

# Simulate 100 inference requests with profiling
print("\n🔍 Profiling 100 inference requests...")

for i in range(100):
    # Stage 1: Data preprocessing
    with profiler.profile_stage("data_preprocessing"):
        # Simulate feature normalization, validation
        X_request = X_test_w[i:i+1].copy()
        X_normalized = (X_request - X_request.mean()) / (X_request.std() + 1e-8)
        time.sleep(0.002 + np.random.uniform(0, 0.003))
    
    # Stage 2: Feature engineering (spatial correlation)
    with profiler.profile_stage("feature_engineering"):
        # Simulate expensive spatial neighbor search
        time.sleep(0.015 + np.random.uniform(0, 0.010))
    
    # Stage 3: Model inference
    with profiler.profile_stage("model_inference"):
        prediction = wafer_model.predict_proba(X_normalized)
        time.sleep(0.001 + np.random.uniform(0, 0.002))
    
    # Stage 4: Post-processing
    with profiler.profile_stage("post_processing"):
        # Simulate result formatting, logging
        result = {
            'prediction': int(prediction.argmax()),
            'confidence': float(prediction.max()),
            'timestamp': time.time()
        }
        time.sleep(0.001 + np.random.uniform(0, 0.001))

# Print summary
profiler.print_summary(sla_ms=25.0)

# Visualize
profiler.plot_latency_distribution()

# Optimization recommendations
print("\n" + "=" * 80)
print("🎯 Optimization Recommendations:")
print("-" * 80)

stats = profiler.get_statistics()
bottleneck = max(stats.items(), key=lambda x: x[1]['mean_ms'])

print(f"\n1. BOTTLENECK IDENTIFIED: '{bottleneck[0]}' ({bottleneck[1]['mean_ms']:.2f}ms mean)")
print(f"   → This stage consumes {bottleneck[1]['mean_ms'] / sum(s['mean_ms'] for s in stats.values()) * 100:.1f}% of total time")

if 'feature_engineering' in stats:
    print(f"\n2. FEATURE ENGINEERING OPTIMIZATION:")
    print(f"   Current: {stats['feature_engineering']['mean_ms']:.2f}ms (spatial neighbor search)")
    print(f"   Recommended: Use KD-tree or spatial indexing → reduce to <5ms (3x speedup)")

print(f"\n3. BATCHING OPPORTUNITY:")
print(f"   Current: Single request = {sum(s['mean_ms'] for s in stats.values()):.2f}ms")
print(f"   Recommended: Batch 32 requests → amortize overhead → 5-10x throughput increase")

print(f"\n4. CACHING OPPORTUNITY:")
print(f"   Cache preprocessed features for repeated requests")
print(f"   Expected: 40-60% latency reduction for cache hits")

print("\n" + "=" * 80)

## 5. 🔎 Error Analysis and Root Cause Detection

### 📝 What's Happening in This Section?

**Purpose:** Build systematic error analysis framework to identify patterns in model failures, prioritize debugging efforts, and detect root causes (data quality issues, feature engineering bugs, model limitations).

**Key Points:**
- **Error clustering**: Group similar failures (spatial patterns, feature value ranges, temporal trends)
- **Confusion matrix analysis**: Identify which classes are confused (Fail predicted as Pass vs Pass as Fail)
- **Feature correlation with errors**: Which feature values predict mistakes? (errors when Vdd < 1.18V)
- **Temporal error patterns**: Are errors increasing over time? (concept drift detection)
- **Severity-based prioritization**: Focus on high-impact errors (false negatives in safety-critical systems)

**Why This Matters:**
- **Faster debugging:** Identify root cause in 15 minutes (not 8 hours of manual inspection)
- **Prioritize fixes:** Focus on errors that impact business (false negatives cost $10K each)
- **Prevent recurrence:** Fix root cause (data quality issue), not symptoms (retrain model)
- **Improve model:** Learn from failures, add features to address systematic errors

**Post-Silicon Application:** Analyze 100 test failures: 45% have Vdd < 1.18V (out-of-spec), 30% have high spatial correlation (neighboring devices failed), 25% have temperature extremes → prioritize Vdd validation in data pipeline (prevents 45% of errors)

In [ ]:
class ErrorAnalyzer:
    """Systematic error analysis framework for ML models"""
    
    def __init__(self, feature_names: List[str]):
        self.feature_names = feature_names
        self.errors = []  # List of error records
    
    def log_error(self, X: np.ndarray, y_true: int, y_pred: int, 
                  prediction_prob: float, metadata: Optional[Dict] = None):
        """
        Log a prediction error
        
        Args:
            X: Feature values
            y_true: True label
            y_pred: Predicted label
            prediction_prob: Prediction probability/confidence
            metadata: Optional metadata (timestamp, device_id, etc.)
        """
        error_record = {
            'features': dict(zip(self.feature_names, X)),
            'y_true': y_true,
            'y_pred': y_pred,
            'prediction_prob': prediction_prob,
            'error_type': self._classify_error(y_true, y_pred),
            'metadata': metadata or {}
        }
        self.errors.append(error_record)
    
    def _classify_error(self, y_true: int, y_pred: int) -> str:
        """Classify error type"""
        if y_true == 0 and y_pred == 1:
            return 'false_positive'
        elif y_true == 1 and y_pred == 0:
            return 'false_negative'
        else:
            return 'correct'
    
    def analyze_error_patterns(self) -> Dict[str, Any]:
        """
        Analyze patterns in errors
        
        Returns:
            Dictionary with error analysis results
        """
        if not self.errors:
            return {'message': 'No errors logged'}
        
        # Error type distribution
        error_types = {}
        for err in self.errors:
            et = err['error_type']
            error_types[et] = error_types.get(et, 0) + 1
        
        # Feature statistics for errors
        feature_stats = {}
        for fname in self.feature_names:
            values = [err['features'][fname] for err in self.errors]
            feature_stats[fname] = {
                'mean': np.mean(values),
                'std': np.std(values),
                'min': np.min(values),
                'max': np.max(values),
                'p25': np.percentile(values, 25),
                'p75': np.percentile(values, 75)
            }
        
        # Confidence distribution for errors
        confidences = [err['prediction_prob'] for err in self.errors]
        
        return {
            'total_errors': len(self.errors),
            'error_type_distribution': error_types,
            'feature_statistics': feature_stats,
            'confidence_stats': {
                'mean': np.mean(confidences),
                'std': np.std(confidences),
                'p50': np.percentile(confidences, 50)
            }
        }
    
    def find_root_causes(self, X_train: np.ndarray, threshold_std: float = 2.0) -> List[Dict[str, Any]]:
        """
        Identify potential root causes by comparing error features to training distribution
        
        Args:
            X_train: Training data for baseline distribution
            threshold_std: Number of std devs for outlier detection
        
        Returns:
            List of potential root causes
        """
        root_causes = []
        
        # Compute training statistics
        train_stats = {}
        for i, fname in enumerate(self.feature_names):
            train_stats[fname] = {
                'mean': X_train[:, i].mean(),
                'std': X_train[:, i].std()
            }
        
        # Check each feature for distribution shift in errors
        for fname in self.feature_names:
            error_values = [err['features'][fname] for err in self.errors]
            error_mean = np.mean(error_values)
            
            # Calculate shift in standard deviations
            train_mean = train_stats[fname]['mean']
            train_std = train_stats[fname]['std']
            
            shift_stds = abs(error_mean - train_mean) / (train_std + 1e-8)
            
            if shift_stds > threshold_std:
                root_causes.append({
                    'feature': fname,
                    'shift_std_devs': shift_stds,
                    'train_mean': train_mean,
                    'error_mean': error_mean,
                    'recommendation': f"Errors occur when {fname} deviates from training distribution"
                })
        
        # Sort by magnitude of shift
        root_causes.sort(key=lambda x: x['shift_std_devs'], reverse=True)
        
        return root_causes
    
    def plot_error_analysis(self):
        """Visualize error patterns"""
        if not self.errors:
            print("No errors to analyze")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # Plot 1: Error type distribution
        ax1 = axes[0, 0]
        analysis = self.analyze_error_patterns()
        error_types = analysis['error_type_distribution']
        
        ax1.bar(error_types.keys(), error_types.values(), color=['red', 'orange'], alpha=0.7)
        ax1.set_ylabel('Count', fontsize=11)
        ax1.set_title('Error Type Distribution', fontsize=12, fontweight='bold')
        ax1.grid(axis='y', alpha=0.3)
        
        # Plot 2: Feature value distributions (errors vs overall)
        ax2 = axes[0, 1]
        
        # Select most important feature (highest variance in errors)
        variances = {fname: np.var([err['features'][fname] for err in self.errors]) 
                    for fname in self.feature_names}
        top_feature = max(variances.items(), key=lambda x: x[1])[0]
        
        error_vals = [err['features'][top_feature] for err in self.errors]
        ax2.hist(error_vals, bins=20, alpha=0.7, color='red', label='Errors')
        ax2.set_xlabel(top_feature, fontsize=11)
        ax2.set_ylabel('Frequency', fontsize=11)
        ax2.set_title(f'Feature Distribution: {top_feature}', fontsize=12, fontweight='bold')
        ax2.legend()
        ax2.grid(axis='y', alpha=0.3)
        
        # Plot 3: Confidence distribution for errors
        ax3 = axes[1, 0]
        confidences = [err['prediction_prob'] for err in self.errors]
        
        ax3.hist(confidences, bins=20, alpha=0.7, color='orange', edgecolor='black')
        ax3.axvline(np.mean(confidences), color='red', linestyle='--', 
                   label=f'Mean: {np.mean(confidences):.3f}')
        ax3.set_xlabel('Prediction Confidence', fontsize=11)
        ax3.set_ylabel('Frequency', fontsize=11)
        ax3.set_title('Confidence Distribution for Errors', fontsize=12, fontweight='bold')
        ax3.legend()
        ax3.grid(axis='y', alpha=0.3)
        
        # Plot 4: Feature importance for errors (variance-based)
        ax4 = axes[1, 1]
        
        sorted_features = sorted(variances.items(), key=lambda x: x[1], reverse=True)[:8]
        features = [f[0] for f in sorted_features]
        vars = [f[1] for f in sorted_features]
        
        ax4.barh(features, vars, color='skyblue', alpha=0.8)
        ax4.set_xlabel('Variance in Error Cases', fontsize=11)
        ax4.set_title('Feature Variance in Errors (High = Discriminative)', 
                     fontsize=12, fontweight='bold')
        ax4.grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plt.show()


# Example: Error analysis for wafer binning model
print("=" * 60)
print("Error Analysis: Wafer Device Binning Failures")
print("=" * 60)

# Generate predictions and log errors
analyzer = ErrorAnalyzer(feature_names)

print("\n🔍 Analyzing model predictions and logging errors...")

y_pred_test = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Log errors
error_count = 0
for i in range(len(X_test)):
    if y_pred_test[i] != y_test[i]:
        analyzer.log_error(
            X=X_test[i],
            y_true=y_test[i],
            y_pred=y_pred_test[i],
            prediction_prob=y_pred_proba[i],
            metadata={'sample_index': i}
        )
        error_count += 1

print(f"✅ Logged {error_count} errors out of {len(X_test)} predictions ({error_count/len(X_test)*100:.1f}% error rate)")

# Analyze patterns
print("\n" + "=" * 60)
print("Error Pattern Analysis")
print("=" * 60)

analysis = analyzer.analyze_error_patterns()

print(f"\nTotal Errors: {analysis['total_errors']}")
print("\nError Type Distribution:")
for etype, count in analysis['error_type_distribution'].items():
    pct = count / analysis['total_errors'] * 100
    print(f"  {etype}: {count} ({pct:.1f}%)")

print("\nFeature Statistics in Error Cases:")
for fname, stats in list(analysis['feature_statistics'].items())[:5]:
    print(f"  {fname}:")
    print(f"    Mean: {stats['mean']:.4f}, Std: {stats['std']:.4f}")
    print(f"    Range: [{stats['min']:.4f}, {stats['max']:.4f}]")

# Find root causes
print("\n" + "=" * 60)
print("Root Cause Detection")
print("=" * 60)

root_causes = analyzer.find_root_causes(X_train, threshold_std=1.5)

if root_causes:
    print(f"\n🎯 Identified {len(root_causes)} potential root causes:\n")
    
    for i, cause in enumerate(root_causes[:5], 1):
        print(f"{i}. Feature: {cause['feature']}")
        print(f"   Shift: {cause['shift_std_devs']:.2f} standard deviations")
        print(f"   Training mean: {cause['train_mean']:.4f}")
        print(f"   Error mean: {cause['error_mean']:.4f}")
        print(f"   💡 {cause['recommendation']}\n")
else:
    print("No significant root causes detected (errors appear random)")

# Visualize
analyzer.plot_error_analysis()

# Recommendations
print("\n" + "=" * 60)
print("🔧 Debugging Recommendations:")
print("-" * 60)

if root_causes:
    top_cause = root_causes[0]
    print(f"\n1. PRIORITY: Investigate '{top_cause['feature']}' feature")
    print(f"   → Errors have {top_cause['shift_std_devs']:.1f}x higher deviation than training")
    print(f"   → Check data pipeline for '{top_cause['feature']}' (quality issues?)")
    
print(f"\n2. ERROR TYPE FOCUS:")
fp_count = analysis['error_type_distribution'].get('false_positive', 0)
fn_count = analysis['error_type_distribution'].get('false_negative', 0)

if fp_count > fn_count:
    print(f"   → More false positives ({fp_count}) than false negatives ({fn_count})")
    print(f"   → Model is too conservative (predicts Fail when actually Pass)")
    print(f"   → Recommendation: Adjust decision threshold upward (0.5 → 0.6)")
else:
    print(f"   → More false negatives ({fn_count}) than false positives ({fp_count})")
    print(f"   → Model is too aggressive (predicts Pass when actually Fail)")
    print(f"   → Recommendation: Adjust decision threshold downward (0.5 → 0.4)")

print(f"\n3. CONFIDENCE ANALYSIS:")
conf_mean = analysis['confidence_stats']['mean']
if conf_mean < 0.7:
    print(f"   → Low average confidence ({conf_mean:.3f}) in error cases")
    print(f"   → Model is uncertain → add more features or collect more training data")
else:
    print(f"   → High average confidence ({conf_mean:.3f}) despite errors")
    print(f"   → Model is overconfident → check for overfitting or feature leakage")

print("\n" + "=" * 60)

## 6. 🚀 Real-World Project Templates

---

### Project 1: Distributed Tracing for Wafer Test Pipeline

**Objective:** Implement end-to-end distributed tracing for STDF data ingestion → feature engineering → binning prediction → result storage pipeline

**Business Value:**
- **Debug time reduction:** 8 hours (manual log inspection) → 15 minutes (trace analysis)
- **SLA compliance:** Identify 80% bottleneck (spatial neighbor search) → optimize → meet <100ms p99 latency
- **Root cause visibility:** Trace errors back to source (STDF parsing failure → which file, which device)

**Features to Implement:**
- Trace ID propagation across distributed services (ingestion → feature store → model service → storage)
- Span instrumentation for each pipeline stage (capture timing, metadata, errors)
- Trace aggregation dashboard (visualize end-to-end flow, identify slowest spans)
- Error correlation (trace ID → all related logs/metrics across services)
- Latency percentile tracking (p50, p95, p99 per stage)
- Context enrichment (wafer_id, device_count, cache hits, model version)

**Success Criteria:**
- ✅ 100% of requests have trace IDs (enable full request tracking)
- ✅ Latency breakdown accurate to ±5ms (identify true bottlenecks)
- ✅ Error traces available within 1 second (real-time debugging)
- ✅ Trace retention: 7 days (support historical analysis)
- ✅ Dashboard visualizes critical path (Gantt chart of spans)

**STDF Data Application:**
- Trace: STDF file upload → parsing → device extraction → feature computation (spatial correlation, parametric aggregations) → model inference → binning decision → DynamoDB write
- Insight: "Why did wafer W0042 take 2 seconds?" → Trace shows spatial correlation took 1.5s → optimize neighbor search

---

### Project 2: SHAP-Based Model Explainability for Fraud Detection

**Objective:** Build production explainability system for fraud detection model, providing real-time SHAP explanations for flagged transactions

**Business Value:**
- **Customer transparency:** Explain why transaction flagged (dispute resolution, regulatory compliance)
- **Model debugging:** Identify when model relies on spurious features (debug before production deployment)
- **Feature engineering:** Remove uninformative features (reduce model size, improve latency)

**Features to Implement:**
- Real-time SHAP value computation (<50ms p99 latency for online explanations)
- Waterfall plot generation (visualize feature contributions for customer service reps)
- Global feature importance aggregation (which features matter most across all predictions)
- Explanation caching (cache SHAP values for repeated requests)
- Explainability API (REST endpoint: POST /explain with transaction_id)

**Success Criteria:**
- ✅ SHAP values computed in <50ms p99 (production SLA)
- ✅ Explanation accuracy: base_value + Σ(SHAP) = prediction (within 0.01)
- ✅ Cover 100% of flagged transactions (regulatory requirement)
- ✅ Customer satisfaction: 80% of disputes resolved with explanation
- ✅ Feature audit: Identify top 10 drivers (focus feature engineering efforts)

**Data Application:**
- Features: transaction_amount, user_account_age, recent_transactions_30d, merchant_category, device_fingerprint
- Explain: "Why flagged?" → SHAP shows transaction_amount (+0.25), recent_transactions_30d (+0.15), device_fingerprint (+0.10) → total +0.50 → flagged

---

### Project 3: Performance Profiling for Recommendation Engine

**Objective:** Profile recommendation engine to identify latency bottlenecks, optimize to meet <100ms p99 latency SLA for 10K QPS

**Business Value:**
- **Revenue impact:** Reduce latency 150ms → 80ms → improve user engagement (click-through rate +5%)
- **Cost optimization:** Identify 10x throughput opportunity (batching) → reduce server count 50% → save $200K/year
- **SLA compliance:** Meet <100ms p99 requirement (avoid penalties, customer churn)

**Features to Implement:**
- Stage-level profiling (user feature fetch, item feature fetch, model scoring, ranking, personalization)
- Percentile latency tracking (p50, p95, p99, p99.9 per stage)
- Batch vs single inference comparison (measure throughput vs latency tradeoff)
- Memory profiling (detect memory leaks, optimize batch size for GPU)
- Continuous profiling (sample 1% of requests, low overhead)
- Bottleneck alerting (alert when single stage >40% of total time)

**Success Criteria:**
- ✅ p99 latency <100ms (meet SLA, currently 150ms)
- ✅ Throughput 10K QPS (support peak traffic)
- ✅ Memory usage <2GB per replica (enable efficient scaling)
- ✅ Identify bottleneck within 5 minutes of deployment (rapid debugging)
- ✅ Optimization targets: 3 highest-impact stages (prioritize engineering effort)

**Data Application:**
- Profile: User features (5ms) → Item features (40ms) → Model scoring (30ms) → Ranking (60ms) → Total 135ms
- Bottleneck: Ranking stage (60ms, 44% of total) → optimize with approximate nearest neighbor → reduce to 15ms → total 90ms (✅ meet SLA)

---

### Project 4: Error Analysis for Autonomous Driving Perception

**Objective:** Build error analysis system for object detection model (safety-critical), identify root causes of false negatives (missed pedestrians)

**Business Value:**
- **Safety:** Reduce false negatives 100 → 10 (prevent accidents, save lives)
- **Regulatory compliance:** Demonstrate systematic error analysis for NHTSA review
- **Model improvement:** Prioritize data collection (focus on challenging scenarios: night, rain, occlusion)

**Features to Implement:**
- Error logging (log every false negative with full context: image, features, metadata)
- Error clustering (group similar errors: nighttime, occlusions, small objects, far distance)
- Feature correlation analysis (which sensor features predict errors?)
- Temporal error tracking (are errors increasing? concept drift?)
- Severity-based prioritization (focus on high-risk errors: highway > parking lot)
- Automated root cause detection (compare error distribution vs training distribution)

**Success Criteria:**
- ✅ 100% false negative logging (capture every missed pedestrian)
- ✅ Root cause identified in <1 hour (rapid debugging for safety issues)
- ✅ Error rate reduction: 2% → 0.5% (systematic improvement)
- ✅ Cluster quality: 80% of errors explained by top 3 clusters (actionable insights)
- ✅ Data collection priorities: top 5 scenarios for new data (efficient improvement)

**Data Application:**
- Analyze: 100 false negatives (missed pedestrians)
- Findings: 40% nighttime (low visibility), 35% occluded (behind cars), 25% small/far (sensor resolution)
- Action: Collect 10K nighttime images, train occlusion-robust model, improve small object detection

---

### Project 5: Observability Dashboard for Yield Prediction Model

**Objective:** Build comprehensive observability dashboard for wafer yield prediction, unifying tracing, explainability, profiling, and error analysis

**Business Value:**
- **Single pane of glass:** Debug any issue from one dashboard (reduce MTTR 2 hours → 20 minutes)
- **Proactive monitoring:** Detect issues before business impact (drift, latency spikes, error patterns)
- **Stakeholder transparency:** Explain model behavior to non-technical fab managers

**Features to Implement:**
- Real-time metrics (QPS, latency percentiles, error rate, model accuracy)
- Distributed traces (visualize end-to-end request flow)
- SHAP waterfall plots (explain specific predictions on-demand)
- Error analysis (cluster errors, identify root causes)
- Performance breakdown (latency by stage, bottleneck identification)
- Drift detection (feature drift, concept drift, performance degradation)
- Alerting (integrate with PagerDuty, Slack for critical issues)

**Success Criteria:**
- ✅ Dashboard load time <2 seconds (real-time debugging)
- ✅ Cover all observability dimensions (metrics, traces, logs, explanations)
- ✅ Drill-down capability (dashboard → trace → error → SHAP explanation in <10 clicks)
- ✅ Adoption: 100% of engineers use for debugging (replace manual log inspection)
- ✅ MTTR reduction: 2 hours → 20 minutes (10x faster debugging)

**STDF Application:**
- Dashboard shows: 
  - Metrics: 500 QPS, 45ms p99 latency, 0.1% error rate, 94% accuracy
  - Trace: STDF ingestion (10ms) → features (25ms) → model (8ms) → storage (2ms)
  - SHAP: Top feature = neighbor_yield_avg (-0.3 → predict low yield)
  - Errors: 10 false positives (predicted low yield, actual high) → cluster shows all have recent process change

---

### Project 6: Latency Optimization for Real-Time Bidding (RTB)

**Objective:** Optimize RTB model inference to meet <10ms p99 latency (bid requests timeout at 100ms, model must be <10% of budget)

**Business Value:**
- **Revenue:** Reduce timeouts 20% → 2% → capture $5M additional revenue/year
- **Competitive advantage:** Faster bids win auctions (improve win rate 30% → 35%)
- **Cost efficiency:** Meet SLA with 50% fewer servers (save $300K/year)

**Features to Implement:**
- Micro-profiling (nanosecond-level timing for critical path)
- Model optimization (quantization, pruning, distillation)
- Feature caching (pre-compute expensive aggregations)
- Batch inference (process multiple bids in parallel)
- GPU vs CPU comparison (measure cost/performance tradeoff)
- Load testing (measure latency degradation under high QPS)

**Success Criteria:**
- ✅ p99 latency <10ms (currently 25ms, 60% reduction needed)
- ✅ Throughput 100K QPS (support peak traffic)
- ✅ Timeout rate <2% (currently 20%, 10x reduction)
- ✅ Cost per inference <$0.0001 (enable profitability at scale)
- ✅ Model accuracy degradation <1% after optimization (maintain performance)

**Data Application:**
- Current: Feature fetch (15ms) + model (8ms) + post-process (2ms) = 25ms p99
- Optimized: Cache features (3ms) + quantized model (4ms) + batch post-process (1ms) = 8ms p99 (✅ meet SLA)

---

### Project 7: Root Cause Detection for Test Failure Clustering (STDF)

**Objective:** Automatically cluster test failures from STDF data, identify root causes (equipment drift, process variation, spatial patterns)

**Business Value:**
- **Debug time:** 8 hours (manual inspection) → 30 minutes (automated clustering)
- **Yield improvement:** Identify systematic issues (equipment calibration) → fix → improve yield 92% → 95% (+$2M/year)
- **Preventive maintenance:** Detect equipment drift early → schedule maintenance → prevent catastrophic failures

**Features to Implement:**
- Failure clustering (group devices by failure signature: which tests failed, values)
- Spatial correlation analysis (wafer map visualization, identify spatial patterns)
- Temporal trend detection (are failures increasing over time?)
- Equipment correlation (which test equipment associated with failures?)
- Parametric outlier detection (which parametric values out-of-spec?)
- Automated root cause ranking (top 5 most likely causes with confidence scores)

**Success Criteria:**
- ✅ Cluster 95% of failures into <10 clusters (actionable categories)
- ✅ Root cause accuracy 80% (validated by fab engineers)
- ✅ Analysis time <30 minutes for 10K device failures (scalable)
- ✅ Spatial pattern detection 90% accurate (identify die location issues)
- ✅ Prevent 50% of future failures (proactive equipment maintenance)

**STDF Application:**
- Input: 1000 failed devices from wafer W0100
- Clustering: Cluster 1 (450 devices, Vdd < 1.18V, spatial: edge dies), Cluster 2 (300 devices, Idd > 180mA, equipment: tester #3), Cluster 3 (250 devices, temperature > 120°C, temporal: last 2 hours)
- Root causes: Edge die yield issue (process), Tester #3 calibration drift (equipment), Thermal chamber malfunction (equipment)

---

### Project 8: Explainability for Medical Diagnosis Model (Regulatory Compliance)

**Objective:** Provide full explainability for medical diagnosis model to meet FDA requirements (510(k) submission)

**Business Value:**
- **Regulatory approval:** FDA requires explainability for AI/ML medical devices (enable $50M market)
- **Clinical trust:** Doctors understand model reasoning → increase adoption 30% → 80%
- **Legal protection:** Audit trail for decisions (defend against malpractice claims)

**Features to Implement:**
- SHAP/LIME explanations for every prediction (waterfall plots, feature attributions)
- Confidence intervals (uncertainty quantification for risk assessment)
- Counterfactual explanations ("What would need to change for different diagnosis?")
- Explanation versioning (link explanation to model version for audit trail)
- Clinical validation (compare SHAP attributions vs clinician reasoning)
- Regulatory report generation (automated PDF with explanation, confidence, model version)

**Success Criteria:**
- ✅ 100% prediction coverage (explainability for every diagnosis)
- ✅ FDA submission includes explainability documentation (510(k) approval)
- ✅ Clinical validation: 85% agreement (SHAP matches clinician reasoning)
- ✅ Explanation generation <1 second (real-time during consultation)
- ✅ Audit trail: 5-year retention (compliance requirement)

**Data Application:**
- Diagnosis: Diabetic retinopathy (grade 3, proliferative)
- SHAP explanation: Microaneurysms (+0.35), neovascularization (+0.28), hemorrhages (+0.15) → total +0.78 → grade 3
- Counterfactual: "If microaneurysms reduced by 50%, prediction would be grade 2 (moderate)"
- Regulatory: PDF report with retinal image, SHAP waterfall, confidence intervals, model version v2.3, timestamp, patient consent

## 7. 🎯 Comprehensive Takeaways: Mastering ML Observability & Debugging

---

### 1. **The Three Pillars of ML Observability**

**Traditional Software Observability:**
- **Metrics:** CPU, memory, QPS, latency, error rate
- **Logs:** Application logs, system logs, access logs
- **Traces:** Distributed request flow across services

**ML Observability Additions:**
- **Model Performance:** Accuracy, precision, recall, drift detection
- **Prediction Explainability:** Feature attributions, SHAP values, confidence
- **Data Quality:** Schema validation, distribution shifts, anomalies

**Unified Approach:**
```
Every ML prediction should generate:
1. Metrics: latency, throughput, cache hit rate
2. Logs: prediction, features, model version, timestamp
3. Traces: request ID, span IDs for each pipeline stage
4. Model telemetry: SHAP values, confidence, drift metrics
5. Data quality: feature validation results, outlier flags
```

**Why All Five:**
- **Metrics alone:** "Latency is 150ms" (but why? which stage? optimization target?)
- **Metrics + Traces:** "Feature fetch is 100ms" (but why slow? cache miss? data size?)
- **Metrics + Traces + Logs:** "Cache miss for user_123" (but why predict wrong? features OK?)
- **Full observability:** "Cache miss → stale features → wrong prediction (SHAP shows outdated purchase_30d)"

---

### 2. **Distributed Tracing for ML Pipelines**

**Trace Anatomy:**
```
Trace ID: req_12345 (single request, end-to-end)
├─ Span 1: request_handler (parent, total: 120ms)
│  ├─ Span 2: feature_fetch (child of 1, 60ms)
│  │  ├─ Span 3: cache_lookup (child of 2, 5ms, HIT)
│  │  └─ Span 4: db_query (child of 2, 50ms, fallback for misses)
│  ├─ Span 5: feature_transform (child of 1, 20ms)
│  ├─ Span 6: model_inference (child of 1, 30ms)
│  └─ Span 7: response_format (child of 1, 10ms)
```

**Critical Metadata to Attach:**
- **Input size:** `{"num_features": 100, "batch_size": 32}`
- **Cache status:** `{"cache_hit": true, "cache_ttl_seconds": 3600}`
- **Model metadata:** `{"model_version": "v2.3", "model_type": "RandomForest"}`
- **Resource usage:** `{"memory_mb": 250, "cpu_cores": 2}`
- **Errors:** `{"error": "NullPointerException", "stack_trace": "..."}`

**Trace Sampling:**
- **Production:** Sample 1-5% (low overhead, sufficient for debugging)
- **Debugging:** Sample 100% temporarily (full visibility, diagnose rare issues)
- **Head-based sampling:** Decide at request start (consistent for entire trace)
- **Tail-based sampling:** Decide at request end (keep only slow/error traces)

**Post-Silicon Example:**
```
Trace: Wafer W0042 binning prediction (total: 2.1 seconds, SLA: <100ms)
├─ STDF parsing: 50ms (✅ fast)
├─ Feature engineering: 1850ms (❌ bottleneck!)
│  ├─ Device aggregation: 100ms
│  └─ Spatial correlation: 1750ms (⚠️ optimize neighbor search!)
├─ Model inference: 150ms
└─ Storage write: 50ms

Action: Optimize spatial correlation (KD-tree index) → reduce 1750ms → 80ms → total 330ms
```

---

### 3. **SHAP Values: The Gold Standard for Explainability**

**What are SHAP Values?**
- **Game theory:** Based on Shapley values (cooperative game theory, fair contribution)
- **Additive:** prediction = base_value + Σ(SHAP_values) (exact decomposition)
- **Model-agnostic:** Works for any model (trees, neural nets, linear models)
- **Consistent:** If feature contribution increases, SHAP value increases (monotonic)

**Mathematical Formulation:**
```
For feature i:
SHAP_i = Σ [|S|! (M - |S| - 1)! / M!] × [f(S ∪ {i}) - f(S)]
        S⊆F\{i}

Where:
- F: all features
- S: subset of features (excluding i)
- f(S): model prediction using only features in S
- M: total number of features

Interpretation: Average marginal contribution of feature i across all possible feature subsets
```

**Practical Computation:**
- **Exact (small models):** Enumerate all 2^M subsets (exponential, only feasible for M<15)
- **Approximate (large models):** Sample subsets, estimate SHAP values (fast, slight error)
- **TreeSHAP (tree models):** Polynomial time algorithm for tree-based models (fast + exact)

**Use Cases:**

**1. Debug Individual Predictions:**
```
Model predicted: Fail (prob = 0.85)
SHAP waterfall:
  Base value: 0.20 (average prediction on training data)
  + vdd = 1.15V: +0.30 (low voltage → fail)
  + neighbor_yield = 0.65: +0.20 (neighbors failed → fail)
  + idd = 105mA: +0.10 (high current → fail)
  + temperature = 28°C: +0.05 (normal temp, slight contribution)
  = Prediction: 0.85 (✅ decomposition exact)

Insight: Primary driver is low Vdd (1.15V vs spec 1.20V ± 0.02V)
```

**2. Global Feature Importance:**
```
Aggregate |SHAP| across all predictions:
1. vdd: 0.25 (most important, check voltage regulation)
2. neighbor_yield: 0.18 (spatial correlation matters)
3. idd: 0.12 (current matters)
4. frequency: 0.02 (least important, consider removing)
```

**3. Feature Engineering Validation:**
```
Added new feature: neighbor_yield_avg (spatial correlation)
SHAP importance: 0.18 (2nd most important feature!)
Validation: Feature is informative, keep in production
```

---

### 4. **Performance Profiling Best Practices**

**Latency Breakdown Strategy:**

**Stage-Level Profiling:**
```python
with profiler.profile_stage("feature_engineering"):
    features = compute_features(input_data)
    # Profiler automatically records duration
```

**Function-Level Profiling (Granular):**
```python
@profile_function
def compute_spatial_correlation(x, y, radius):
    # Detailed profiling of expensive function
    neighbors = find_neighbors(x, y, radius)  # Profile this separately
    return np.mean([n.yield_pct for n in neighbors])
```

**Line-Level Profiling (Debugging):**
```python
# Use line_profiler for hotspot identification
@profile
def expensive_function():
    data = load_data()        # Line 1: 50ms
    features = transform(data)  # Line 2: 200ms ← bottleneck!
    return features
```

**Percentile Analysis:**
```
Latency distribution:
- Mean: 45ms (misleading, doesn't show tail)
- P50: 40ms (median, half of requests faster)
- P95: 80ms (5% of requests slower)
- P99: 150ms (1% of requests, tail latency)
- P99.9: 500ms (rare, but impacts user experience)

SLA: p99 <100ms
Status: ❌ FAIL (150ms p99)
Action: Investigate p95-p99 range (80-150ms), identify outliers
```

**Batch vs Single Inference:**
```
Single inference:
- Latency: 30ms per request
- Throughput: 33 QPS (1000ms / 30ms)
- Overhead: High (model load, context switch per request)

Batch inference (batch_size=32):
- Latency: 80ms for 32 requests (2.5ms per request amortized)
- Throughput: 400 QPS (32 requests / 80ms)
- Speedup: 12x throughput (GPU parallelism, reduced overhead)
- Tradeoff: Higher per-request latency (80ms vs 30ms), but much higher total throughput
```

**Memory Profiling:**
```python
import tracemalloc

tracemalloc.start()

# Run inference
predictions = model.predict(large_batch)

current, peak = tracemalloc.get_traced_memory()
print(f"Peak memory: {peak / 1024**2:.1f} MB")

tracemalloc.stop()

# Detect memory leaks:
# If peak memory increases over time → memory leak (investigate object retention)
```

---

### 5. **Error Analysis Frameworks**

**Error Classification:**

**Type 1: False Positive (FP)**
- **Definition:** Predicted positive, actually negative
- **Example:** Predicted Fail, device passed
- **Impact:** Unnecessary rejection (yield loss, revenue loss)
- **Cost:** Low-moderate (can re-test, but wastes time/money)

**Type 2: False Negative (FN)**
- **Definition:** Predicted negative, actually positive
- **Example:** Predicted Pass, device failed in customer hands
- **Impact:** Escape to customer (warranty claims, reputation damage)
- **Cost:** High-critical (safety issues, customer trust loss)

**Asymmetric Cost:**
```
Fraud detection:
- FP: Block legitimate transaction (customer annoyed)
- FN: Miss fraud (lose $1000 per transaction)
- Strategy: Minimize FN (lower threshold, accept more FP)

Post-silicon validation:
- FP: Reject good device (yield loss: $50)
- FN: Ship bad device (warranty claim: $500, reputation damage: priceless)
- Strategy: Minimize FN (strict binning, accept some FP)
```

**Root Cause Detection:**

**Method 1: Feature Distribution Comparison**
```python
# Compare error features vs training features
for feature in features:
    error_mean = np.mean([e[feature] for e in errors])
    train_mean = np.mean(X_train[:, feature])
    shift = abs(error_mean - train_mean) / train_std
    
    if shift > 2.0:
        print(f"Root cause: {feature} shifted {shift:.1f} std devs in errors")
```

**Method 2: Error Clustering**
```python
# Group errors by similarity
from sklearn.cluster import DBSCAN

error_features = np.array([e['features'] for e in errors])
clustering = DBSCAN(eps=0.3, min_samples=5).fit(error_features)

# Largest cluster = most common error pattern
largest_cluster_id = Counter(clustering.labels_).most_common(1)[0][0]
cluster_errors = error_features[clustering.labels_ == largest_cluster_id]

print(f"Cluster {largest_cluster_id}: {len(cluster_errors)} errors")
print(f"Common pattern: {np.mean(cluster_errors, axis=0)}")
```

**Method 3: Decision Tree Error Explainer**
```python
# Train decision tree to predict errors (interpretable)
X_all = np.vstack([X_correct, X_errors])
y_all = [0] * len(X_correct) + [1] * len(X_errors)

error_tree = DecisionTreeClassifier(max_depth=3)
error_tree.fit(X_all, y_all)

# Interpret tree rules
# Example: "Errors occur when vdd < 1.18 AND neighbor_yield < 0.75"
```

---

### 6. **Observability Dashboard Design**

**Dashboard Hierarchy:**

**Level 1: Executive Summary (1 screen)**
```
┌─────────────────────────────────────────────┐
│  ML Model Health Overview                   │
├─────────────────────────────────────────────┤
│  QPS: 5,420          Latency p99: 85ms      │
│  Accuracy: 94.2%     Error Rate: 0.3%       │
│  ✅ All systems operational                 │
└─────────────────────────────────────────────┘
```

**Level 2: Component Breakdown (drill-down)**
```
┌─────────────────────────────────────────────┐
│  Latency Breakdown                          │
├─────────────────────────────────────────────┤
│  Feature Fetch:      30ms (35%)             │
│  Model Inference:    40ms (47%)             │
│  Post-Processing:    15ms (18%)             │
│  ⚠️ Model Inference above target (30ms)     │
└─────────────────────────────────────────────┘
```

**Level 3: Trace/Error Drilldown (debug specific issues)**
```
┌─────────────────────────────────────────────┐
│  Recent Errors (last hour)                  │
├─────────────────────────────────────────────┤
│  12:34:56 - Trace ID: abc123                │
│    Error: NullPointerException              │
│    Feature: neighbor_yield_avg = None       │
│    ⚠️ Click for full trace                  │
└─────────────────────────────────────────────┘
```

**Key Metrics to Display:**

**Real-Time (1-minute window):**
- QPS (queries per second)
- Latency (p50, p95, p99)
- Error rate (% of requests)
- Cache hit rate

**Hourly Aggregates:**
- Accuracy (% correct predictions)
- Prediction distribution (% Pass vs Fail)
- Feature drift (KS test p-value)
- Resource usage (CPU, memory, GPU)

**Daily Aggregates:**
- Model performance trend (accuracy over time)
- Error analysis (top error patterns)
- Cost metrics ($ per 1M predictions)

**Integration with Existing Tools:**
- **Grafana:** Time-series metrics, alerting
- **Jaeger/Zipkin:** Distributed tracing visualization
- **ELK Stack:** Log aggregation, search
- **Custom dashboard:** ML-specific (SHAP, error analysis)

---

### 7. **Debugging Workflow**

**Systematic Debugging Process:**

**Step 1: Reproduce Issue**
```
❌ "Model sometimes predicts wrong"
✅ "Model predicts wrong for wafer W0042, device D123 (trace ID: abc123)"

Gather:
- Trace ID (full request flow)
- Input data (features, raw values)
- Expected output vs actual output
- Timestamp (when did it occur?)
```

**Step 2: Isolate Stage**
```
Use trace to identify which stage failed:
- Feature fetch: OK (30ms, cache hit)
- Feature transform: OK (15ms, no nulls)
- Model inference: ⚠️ ANOMALY (prediction=0.95, expected=0.30)
- Post-processing: OK

Conclusion: Issue in model inference stage
```

**Step 3: Analyze Inputs**
```
Check feature values:
- vdd: 1.15V (⚠️ low, spec is 1.20V ± 0.02V)
- idd: 105mA (OK)
- neighbor_yield: 0.65 (⚠️ low, neighbors failed)

Hypothesis: Low Vdd + low neighbor yield → high fail prediction
```

**Step 4: Explain Prediction (SHAP)**
```
Compute SHAP values:
- vdd: +0.40 (major contributor to fail prediction)
- neighbor_yield: +0.25 (secondary contributor)
- Other features: +0.10 (minor)

Validation: SHAP confirms hypothesis (low Vdd drives prediction)
```

**Step 5: Root Cause**
```
Why is Vdd low?
- Check upstream: STDF file has Vdd=1.15V (data is correct)
- Check equipment: Tester #3 voltage regulator drifted
- Check process: Recent fab change lowered target voltage

Root cause: Equipment calibration drift (Tester #3)
```

**Step 6: Fix + Validate**
```
Fix: Recalibrate Tester #3 voltage regulator
Validate:
- Re-test device D123: Vdd now 1.20V → prediction=0.30 (correct!)
- Monitor: Check all devices tested by Tester #3 (batch revalidation)
```

---

### 8. **Explainability Beyond SHAP**

**LIME (Local Interpretable Model-agnostic Explanations):**
- **Approach:** Fit linear model locally around prediction (interpretable approximation)
- **Use case:** Explain complex models (neural networks) with simple linear features
- **Tradeoff:** Faster than SHAP (approximate), but less theoretically sound

**Counterfactual Explanations:**
- **Question:** "What would need to change for different prediction?"
- **Example:** "If Vdd increased from 1.15V to 1.19V, prediction would change from Fail to Pass"
- **Use case:** Actionable feedback (what to fix to change outcome)

**Anchor Explanations:**
- **Question:** "What features guarantee this prediction (regardless of others)?"
- **Example:** "If Vdd < 1.18V, prediction is Fail with 95% confidence (even if other features change)"
- **Use case:** Robust rules (confidence in explanation)

**Attention Mechanisms (Neural Networks):**
- **Approach:** Visualize which input tokens model focuses on
- **Use case:** Text/image models (which words/pixels most important)
- **Example:** Sentiment analysis: "The food was [great] but service was terrible" (attention on "great" → positive)

**Feature Importance vs Feature Attribution:**
- **Feature Importance:** Global (which features matter overall?)
- **Feature Attribution:** Local (which features matter for this prediction?)
- **Example:** 
  - Importance: Vdd is most important feature globally (25% of model decisions)
  - Attribution: For device D123, neighbor_yield contributed most (+0.30 SHAP) even though globally less important

---

### 9. **Latency Optimization Strategies**

**Low-Hanging Fruit (Quick Wins):**

**1. Caching:**
```python
# Before: Query database every request (50ms)
features = db.query(f"SELECT * FROM features WHERE user_id={user_id}")

# After: Cache in Redis (1ms)
features = cache.get(f"features:{user_id}")
if not features:
    features = db.query(...)
    cache.set(f"features:{user_id}", features, ttl=3600)

# Speedup: 50x (50ms → 1ms for cache hits)
```

**2. Batching:**
```python
# Before: Process 1 request at a time (30ms each)
for request in requests:
    prediction = model.predict(request)  # 30ms × 100 = 3000ms

# After: Batch 32 requests (100ms for 32)
for batch in batched(requests, batch_size=32):
    predictions = model.predict(np.array(batch))  # 100ms for 32 requests

# Speedup: 10x (3000ms → 300ms for 100 requests)
```

**3. Feature Selection:**
```python
# Before: Use all 100 features (40ms preprocessing)
X = preprocess_all_features(input_data)

# After: Use top 20 features by importance (8ms preprocessing)
X = preprocess_selected_features(input_data, top_k=20)

# Speedup: 5x (40ms → 8ms), minimal accuracy loss (<1%)
```

**Advanced Optimizations:**

**4. Model Quantization:**
```python
# Before: FP32 model (200MB, 30ms inference)
model_fp32 = load_model("model_fp32.pkl")

# After: INT8 model (50MB, 10ms inference)
model_int8 = quantize_model(model_fp32, dtype="int8")

# Speedup: 3x (30ms → 10ms), <1% accuracy loss
# Storage: 4x smaller (200MB → 50MB)
```

**5. Model Pruning:**
```python
# Before: 100-tree Random Forest (30ms inference)
model = RandomForestClassifier(n_estimators=100)

# After: 20-tree pruned forest (8ms inference)
model = prune_trees(model, target_trees=20, metric="importance")

# Speedup: 3.75x (30ms → 8ms), <2% accuracy loss
```

**6. Approximate Algorithms:**
```python
# Before: Exact nearest neighbor search (O(n), 200ms)
neighbors = [d for d in devices if distance(d, target) < radius]

# After: Approximate KD-tree (O(log n), 20ms)
neighbors = kd_tree.query_radius(target, radius)

# Speedup: 10x (200ms → 20ms), 99% recall
```

---

### 10. **Error Budget and SLA Management**

**Error Budget Concept:**
```
SLA: 99.9% uptime (3 nines)
Error budget: 0.1% downtime per month
= 43 minutes downtime allowed per month

Current status:
- Incidents this month: 2 outages (10 min + 15 min = 25 min)
- Budget remaining: 43 - 25 = 18 minutes
- ✅ Still within budget (but monitor closely)
```

**Latency SLA:**
```
SLA: p99 latency <100ms
Measurement window: 7 days rolling

Current status:
- Day 1-6: p99 = 85ms (✅ within SLA)
- Day 7: p99 = 120ms (❌ violation)
- Action: Trigger investigation, identify root cause
```

**Accuracy SLA:**
```
SLA: Model accuracy >95% (measured daily)

Current status:
- Baseline accuracy: 96%
- Current accuracy: 94% (2% degradation)
- Alert threshold: 5% degradation
- Status: ⚠️ Warning (monitor, not critical yet)
```

**Burn Rate:**
```
Error budget burn rate = (current error rate) / (allowed error rate)

Example:
- Allowed error rate: 0.1% (from 99.9% SLA)
- Current error rate: 0.5%
- Burn rate: 0.5% / 0.1% = 5x

Interpretation: Burning error budget 5x faster than sustainable
Action: Fix errors immediately (or exhaust budget in 6 days instead of 30)
```

---

### 11. **ML-Specific Monitoring Metrics**

**Beyond Traditional Metrics:**

**Prediction Distribution:**
```python
# Monitor: Are predictions shifting over time?
pred_distribution_week1 = [0.8, 0.2]  # [Pass: 80%, Fail: 20%]
pred_distribution_week2 = [0.6, 0.4]  # [Pass: 60%, Fail: 40%]

# Shift: Fail predictions doubled (20% → 40%)
# Possible causes:
# 1. Concept drift (real change in data)
# 2. Data quality issue (bad features)
# 3. Upstream pipeline bug
```

**Confidence Distribution:**
```python
# Monitor: Is model becoming less confident?
confidence_week1 = [0.95, 0.92, 0.88, ...]  # High confidence
confidence_week2 = [0.65, 0.58, 0.72, ...]  # Low confidence

# Shift: Average confidence dropped (0.92 → 0.65)
# Interpretation: Model uncertain (new data patterns)
# Action: Collect labels, retrain on recent data
```

**Feature Null Rate:**
```python
# Monitor: Are features missing more often?
null_rate_week1 = {'vdd': 0.1%, 'idd': 0.2%, ...}
null_rate_week2 = {'vdd': 5.0%, 'idd': 0.2%, ...}

# Shift: Vdd null rate spiked (0.1% → 5.0%)
# Root cause: Upstream STDF pipeline issue
# Action: Alert data engineering team
```

**Inference Volume:**
```python
# Monitor: Are we getting expected traffic?
expected_qps = 1000
actual_qps_week1 = 980  # ✅ Normal variance
actual_qps_week2 = 200  # ❌ 80% drop!

# Shift: Traffic dropped 80%
# Possible causes:
# 1. Upstream service down (requests not reaching model)
# 2. Client bug (not calling API)
# 3. Business change (fab production paused)
```

---

### 12. **Integration with Existing Tools**

**OpenTelemetry (Unified Observability):**
```python
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.exporter.jaeger import JaegerExporter

# Configure tracing
trace.set_tracer_provider(TracerProvider())
tracer = trace.get_tracer(__name__)

# Instrument ML pipeline
with tracer.start_as_current_span("ml_inference") as span:
    span.set_attribute("model.version", "v2.3")
    span.set_attribute("input.size", len(features))
    
    prediction = model.predict(features)
    
    span.set_attribute("prediction.value", float(prediction))
    span.set_attribute("prediction.confidence", float(confidence))
```

**Prometheus (Metrics):**
```python
from prometheus_client import Counter, Histogram

# Define metrics
prediction_counter = Counter('ml_predictions_total', 'Total predictions', ['model', 'outcome'])
latency_histogram = Histogram('ml_inference_latency_seconds', 'Inference latency')

# Instrument code
with latency_histogram.time():
    prediction = model.predict(features)

prediction_counter.labels(model='yield_model', outcome='pass').inc()
```

**MLflow (Experiment Tracking + Model Registry):**
```python
import mlflow

# Log experiment
with mlflow.start_run():
    mlflow.log_param("n_estimators", 100)
    mlflow.log_metric("accuracy", 0.95)
    mlflow.sklearn.log_model(model, "model")
    
    # Log SHAP explanations
    mlflow.log_artifact("shap_summary.png")
```

---

### 13. **Production Debugging War Stories**

**Case Study 1: The Mysterious Latency Spike**
```
Symptom: p99 latency spiked from 50ms → 500ms
Initial hypothesis: Database slow (checked: DB fine)
Trace analysis: Feature fetch stage 5ms → 400ms
Root cause: Cache eviction (Redis memory full)
Fix: Increase Redis memory + add TTL monitoring
Lesson: Monitor cache metrics (hit rate, memory usage)
```

**Case Study 2: The Silent Model Degradation**
```
Symptom: Customer complaints (more false positives)
Initial hypothesis: Model bug (checked: model unchanged)
Error analysis: FP rate increased 2% → 8% (4x!)
Feature analysis: avg_transaction_30d shifted (distribution drift)
Root cause: Upstream feature pipeline bug (wrong date range)
Fix: Fix feature computation, retrain model
Lesson: Monitor feature distributions, not just accuracy
```

**Case Study 3: The Misleading SHAP Values**
```
Symptom: SHAP shows feature X important, but removing it improves accuracy
Initial hypothesis: SHAP bug (checked: SHAP correct)
Root cause: Feature X correlated with label during training (data leakage)
Fix: Remove feature X, retrain without leakage
Lesson: SHAP shows what model uses, not what's causal (correlation ≠ causation)
```

---

### 14. **Testing Observability Systems**

**Unit Tests:**
```python
def test_tracer_span_creation():
    tracer = MLTracer()
    trace_id = "test_001"
    
    span_id = tracer.start_trace(trace_id, "test_op")
    assert span_id in tracer.active_spans
    
    tracer.end_span(span_id)
    assert span_id not in tracer.active_spans

def test_shap_explainer_decomposition():
    explainer = SHAPExplainer(model, X_background)
    shap_values = explainer.explain_instance(X_test[0])
    
    # Verify: base + sum(SHAP) ≈ prediction
    reconstructed = explainer.base_value + sum(shap_values.values())
    actual_pred = model.predict_proba(X_test[0])[0, 1]
    
    assert abs(reconstructed - actual_pred) < 0.01
```

**Integration Tests:**
```python
def test_end_to_end_observability():
    # Simulate full prediction with observability
    tracer = MLTracer()
    profiler = PerformanceProfiler()
    explainer = SHAPExplainer(model, X_train)
    
    trace_id = "integration_test"
    tracer.start_trace(trace_id, "prediction")
    
    with profiler.profile_stage("inference"):
        prediction = model.predict(X_test[0])
    
    shap_values = explainer.explain_instance(X_test[0])
    
    # Verify all components captured data
    assert len(tracer.get_trace(trace_id)) > 0
    assert "inference" in profiler.stage_timings
    assert len(shap_values) == X_test.shape[1]
```

---

### 15. **Key Takeaways Summary**

✅ **Distributed tracing enables end-to-end debugging** - trace request from input → feature → model → output (correlate errors across services)

✅ **SHAP values provide exact feature attribution** - prediction = base + Σ(SHAP) (explain individual predictions, debug model behavior)

✅ **Performance profiling identifies bottlenecks** - measure latency breakdown by stage (optimize highest-impact operations first)

✅ **Error analysis reveals systematic patterns** - cluster errors, compare distributions (prioritize fixes, prevent recurrence)

✅ **Percentile metrics matter more than averages** - p99 latency captures tail behavior (impacts user experience, SLA compliance)

✅ **Observability requires instrumentation** - logs, metrics, traces, SHAP, drift detection (unified view of ML system health)

✅ **Root cause detection requires data** - compare error features vs training distribution (identify shifts, data quality issues)

✅ **Explainability builds trust** - SHAP waterfall plots, confidence intervals (regulatory compliance, stakeholder transparency)

✅ **Post-silicon observability is critical** - STDF pipeline tracing, spatial error clustering, equipment correlation (faster debug, higher yield)

✅ **Production checklist:** Distributed tracing, SHAP on-demand, latency profiling, error logging, drift monitoring, dashboards, alerts

---

### 16. **Production Readiness Checklist**

**Distributed Tracing:**
- [ ] Trace ID generation (UUID per request)
- [ ] Trace ID propagation (HTTP headers, message queues)
- [ ] Span instrumentation (all pipeline stages)
- [ ] Metadata attachment (model version, features, cache status)
- [ ] Trace export (Jaeger, Zipkin, or OpenTelemetry)
- [ ] Trace sampling (1-5% in production, 100% for debugging)

**Model Explainability:**
- [ ] SHAP explainer deployed (on-demand or pre-computed)
- [ ] Explanation API (<50ms p99 latency for online queries)
- [ ] Waterfall plot generation (visualization for stakeholders)
- [ ] Global feature importance (aggregated SHAP across dataset)
- [ ] Explanation versioning (link to model version)
- [ ] Explanation storage (7-day retention for audit)

**Performance Profiling:**
- [ ] Stage-level timing (feature fetch, inference, post-processing)
- [ ] Percentile tracking (p50, p95, p99 per stage)
- [ ] Continuous profiling (1% sampling, low overhead)
- [ ] Profiling dashboard (real-time latency breakdown)
- [ ] Bottleneck alerting (single stage >40% of total time)
- [ ] Resource monitoring (CPU, memory, GPU utilization)

**Error Analysis:**
- [ ] Error logging (100% of prediction errors)
- [ ] Error clustering (group similar failures)
- [ ] Root cause detection (feature distribution comparison)
- [ ] Error analysis dashboard (visualize patterns)
- [ ] Severity classification (FP vs FN, business impact)
- [ ] Error trend monitoring (are errors increasing?)

**Observability Dashboard:**
- [ ] Real-time metrics (QPS, latency, error rate, accuracy)
- [ ] Trace visualization (Gantt chart, span hierarchy)
- [ ] SHAP explanation viewer (on-demand queries)
- [ ] Error analysis view (clusters, root causes)
- [ ] Alert integration (PagerDuty, Slack, email)
- [ ] Mobile access (debug from anywhere)

---

### 17. **Next Steps in Learning**

**Notebook 131: Containerization for ML**
- Docker for model serving (reproducible environments, version control)
- Multi-stage builds (optimize image size)
- Container orchestration basics (Kubernetes, ECS)

**Notebook 132: Service Mesh for ML**
- Istio for traffic management (A/B testing, canary deployment)
- Distributed tracing integration (automatic span generation)
- Observability out-of-the-box (metrics, logs, traces)

**Notebook 133: CI/CD for ML**
- Automated testing (unit tests, integration tests, model validation)
- Continuous training pipelines (trigger retraining on drift)
- Deployment automation (blue-green, canary, rollback)

**Beyond MLOps:**
- **AIOps:** AI for observability (anomaly detection, root cause analysis, auto-remediation)
- **Chaos Engineering:** Test observability under failure conditions (random pod kills, network delays)
- **Edge ML:** Observability for edge devices (limited bandwidth, offline operation)

---

**Congratulations! You've mastered ML observability and debugging systems.** 🎉

You now understand:
- ✅ Distributed tracing (track request flow end-to-end)
- ✅ Model explainability (SHAP values, waterfall plots)
- ✅ Performance profiling (latency breakdown, bottleneck identification)
- ✅ Error analysis (clustering, root cause detection)
- ✅ Production observability (dashboards, alerts, debugging workflows)

**You're now equipped to debug production ML systems with confidence and speed.** 🚀

## 🎯 Key Takeaways

### When to Use ML Observability
- **Production models**: Any model serving predictions in production (100+ requests/day)
- **Critical decisions**: High-cost errors (yield prediction, binning decisions worth $M annually)
- **Data drift monitoring**: Input distributions change over time (new product introductions, process changes)
- **Model performance tracking**: Detect accuracy degradation before business impact
- **Debugging**: Investigate prediction errors, outliers, unexpected behavior

### Limitations
- **Metric selection**: Too many metrics = alert fatigue, too few = miss issues (balance coverage vs. noise)
- **Lag in ground truth**: Can't compute accuracy until labels available (weeks/months for field failures)
- **Computational overhead**: Logging all predictions + features adds latency (5-10ms typical)
- **Storage costs**: Retaining prediction logs for analysis (GB-TB scale for high-volume services)

### Alternatives
- **Manual spot checks**: Periodic manual review of predictions (doesn't scale, misses systemic issues)
- **A/B testing**: Continuous comparison to baseline model (good for improvement validation)
- **Offline evaluation**: Batch model testing on held-out sets (misses production-specific issues)
- **Unit tests**: Test model code correctness (doesn't catch data distribution shifts)

### Best Practices
- **Multi-layer monitoring**: Infrastructure (latency, errors) + model (accuracy, drift) + business (revenue impact)
- **Statistical alerts**: 3-sigma rules, sequential testing (avoid false alarms from noise)
- **Logging strategy**: Sample 1-10% of predictions for detailed analysis, aggregate metrics for all
- **Alerting hierarchy**: P0 (immediate page) for >10% accuracy drop, P1 (ticket) for drift warnings
- **Runbooks**: Document response procedures for each alert type (who investigates? escalation path?)
- **Feedback loops**: Route alerts to data scientists, enable quick model retraining/rollback

## 📊 Diagnostic Checks Summary

### Implementation Checklist
✅ **Logging Infrastructure**
- Structured logging: JSON format with timestamp, model_version, input_features, prediction, latency
- Sampling strategy: 100% for errors, 10% for normal predictions, 1% for detailed feature logging
- Log aggregation: ELK stack (Elasticsearch, Logstash, Kibana) or CloudWatch Logs
- Retention policy: 30 days for detailed logs, 1 year for aggregated metrics

✅ **Metrics Tracking**
- Prediction metrics: Distribution (mean, p50, p95, p99), outlier rates (>3σ)
- Performance metrics: Latency (p50, p95, p99), throughput (requests/sec), error rate
- Data drift: KL divergence, KS test p-values for feature distributions
- Model accuracy: Online metrics (when labels available), proxy metrics (confidence scores)

✅ **Alerting System**
- Statistical alerts: 3-sigma rules for metric deviations, sequential probability ratio test (SPRT)
- Thresholds: P0 (>10% accuracy drop, >50% latency increase), P1 (drift p<0.01, error rate >5%)
- Alert routing: PagerDuty for P0, Slack/email for P1, dashboard for P2
- Deduplication: Suppress duplicate alerts within 1hr window

✅ **Debugging Tools**
- Prediction explainability: SHAP values logged for sampled predictions
- Error analysis: Cluster errors by input characteristics, identify failure modes
- A/B testing: Shadow mode for canary deployments, traffic splitting 90/10
- Rollback mechanism: Automated rollback if accuracy drops >15% for 1hr

### Quality Metrics
- **Logging overhead**: <5ms p95 latency increase, <5% CPU overhead
- **Alert accuracy**: <10% false positive rate, <1% false negative rate
- **Mean time to detect (MTTD)**: <15min for critical issues
- **Mean time to recover (MTTR)**: <2hr from detection to resolution

### Post-Silicon Validation Applications
**1. Yield Prediction Model Observability**
- Metrics: Prediction distribution (yield% mean, std), feature drift (test parameter distributions)
- Alerts: P0 if predicted yield <70% for >100 wafers (investigate immediately), P1 if KS test p<0.01 for Vdd distribution
- Debugging: SHAP values identify which test parameters driving low yield predictions
- Business value: Detect model degradation 12-24hr before manual review, $4M-$8M/year faster response

**2. Binning Model Monitoring (Device Classification)**
- Metrics: Bin distribution (% premium, standard, low-power), confidence scores, misclassification rate (when ground truth available)
- Alerts: P0 if >30% bins with confidence <80% (model uncertainty spike), P1 if bin mix shifts >15% vs. historical
- Root cause: Correlate bin shifts with lot attributes (fab, product, test site)
- Business value: Prevent revenue loss from incorrect binning ($10M-$25M/year at high volumes)

**3. Test Time Prediction Model Debugging**
- Metrics: Prediction error (MAE, RMSE), residual distribution, outlier rate (>2x predicted time)
- Alerts: P1 if MAE increases >20% vs. baseline (model drift or test program changes)
- Debugging: Stratify errors by product/site, identify systematic biases
- Business value: Accurate test time forecasts enable capacity planning, $3M-$8M/year optimized scheduling

### Business ROI Estimation

**Scenario 1: Medium-Volume Semiconductor Fab (100K wafers/year, 5 production models)**
- Model degradation detection: Catch issues 24hr earlier = **$2.5M/year** reduced scrap/rework
- Data drift alerts: Identify test program changes breaking assumptions = **$1.5M/year** avoided bad predictions
- Automated rollback: 2hr MTTR vs. 24hr manual = **$3M/year** reduced downtime impact
- **Total ROI: $7M/year** (cost: $200K observability platform + $150K team = $6.65M net)

**Scenario 2: High-Volume Automotive Semiconductor (500K wafers/year, 20+ models)**
- Comprehensive monitoring: 15min MTTD for all critical models = **$15M/year** faster incident response
- A/B testing infrastructure: Safe canary deployments = **$8M/year** avoided bad model releases
- Explainability logging: SHAP values for error debugging = **$5M/year** faster root cause (4hr → 30min)
- **Total ROI: $28M/year** (cost: $1M enterprise observability + $500K team = $26.5M net)

**Scenario 3: Advanced Node R&D Fab (<10K wafers/year)**
- Experimental model monitoring: Track A/B tests across process experiments = **$3M/year** research velocity
- Feature importance tracking: Identify which process parameters drive predictions = **$2.5M/year** physics insights
- Model versioning + rollback: Quick recovery from bad model updates = **$1.5M/year** avoided experiment delays
- **Total ROI: $7M/year** (cost: $150K observability tools + $100K setup = $6.75M net)

## 📈 Progress Update

**Notebook 130: ML Observability & Debugging** expanded from 11 → 15 cells ✅

**Session progress: 10 notebooks completed**
- 12-cell: 129, 133, 162, 163, 164  
- 11-cell: 111, 112, 116, 130

Next: 138, 151 (11-cell notebooks)

---

## 🎓 Mastery Achievement

**You now have production-grade expertise in:**
- ✅ Implementing structured logging with sampling strategies for production ML models
- ✅ Tracking prediction, performance, and data drift metrics with statistical alerting
- ✅ Building observability dashboards (ELK stack, CloudWatch) with P0/P1/P2 alert hierarchies
- ✅ Debugging model errors with SHAP explainability and error clustering analysis
- ✅ Applying ML observability to yield prediction, binning models, and test time forecasting

**Next Steps:**
- **Advanced Drift Detection**: Multivariate drift (multiple features jointly), context-aware alerting
- **Causal Debugging**: Root cause analysis linking input changes to prediction degradation
- **Automated Remediation**: Self-healing models that retrain on drift detection